# Line Plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how Graphic Features can be modified by slicing!

In [2]:
import fastplotlib as fpl
from fastplotlib.graphics.selectors import Synchronizer

import numpy as np
from sidecar import Sidecar
from ipywidgets import VBox
from itertools import product

### First generate some data.

In [3]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [4]:
# Create a plot instance
plot_l = fpl.Plot(size=(600, 300))

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

sc = Sidecar(title="lines")

with sc: 
    display(plot_l.show())

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### Graphic features support slicing! :D

In [5]:
# indexing of colors
cosine_graphic.colors[:40] = "magenta"

In [6]:
# more complex indexing, set the red value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

### You can capture changes to a graphic feature as events

In [7]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [8]:
# more complex indexing of colors
# from point 15 - 50, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

FeatureEvent @ 0x7efe4bfa4e50
type: colors
pick_info: {'index': range(15, 50, 3), 'collection-index': None, 'world_object': <weakproxy at 0x7efe51c507c0 to Line at 0x7efe4bf27ed0>, 'new_data': array([[0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.]], dtype=float32)}



In [9]:
plot_l.close()
sc.close()

### colormaps

In [10]:
plot = fpl.Plot(size=(600, 300))

plot.add_line(sine, thickness=10, name="sine")

sc = Sidecar(title="line plot")

with sc:
    display(plot.show())

RFBOutputContext()

In [11]:
# set cmap 
plot["sine"].cmap = "jet"

In [12]:
# set cmap values equal sine wave y-values 
plot["sine"].cmap.values = sine[:, 1]

In [13]:
# set cmap values equal to cosine wave y-values
plot["sine"].cmap.values = cosine[:, 1]

In [14]:
# change the cmap, cmap values remain the same
plot["sine"].cmap = "viridis"

**Qualitative data**

In [15]:
# qualitatively set the cmap values
cmap_values = [0] * 25 + [1] * 5 + [2] * 50 + [3] * 20
plot["sine"].cmap.values = cmap_values

**Qualitative cmap**

In [16]:
# now change the cmap to new cmap with those same cmap values
plot["sine"].cmap = "tab10"

In [17]:
plot.close()
sc.close()

## Line Collections

**Generate some data, 4 x 4 circles**

In [18]:
def make_circle(center, radius: float, n_points: int = 75) -> np.ndarray:
    theta = np.linspace(0, 2 * np.pi, n_points)
    xs = radius * np.sin(theta)
    ys = radius * np.cos(theta)

    return np.column_stack([xs, ys]) + center


spatial_dims = (50, 50)

# this makes 16 circles, so we can create 16 cmap values, so it will use these values to set the
# color of the line based by using the cmap as a LUT with the corresponding cmap_value
circles = list()
for center in product(range(0, spatial_dims[0], 15), range(0, spatial_dims[1], 15)):
    circles.append(make_circle(center, 5, n_points=75))

pos_xy = np.vstack(circles)

### Qualitative cmap

Useful for clustering or classification

In [19]:
plot = fpl.Plot(size=(600, 600))

# things like class labels, cluster labels, etc.
cmap_values = [
    0, 1, 1, 2,
    0, 0, 1, 1,
    2, 2, 3, 3,
    1, 1, 1, 5
]

plot.add_line_collection(
    circles, 
    cmap="tab10", 
    cmap_values=cmap_values, 
    thickness=15
)

sc = Sidecar(title="lines collection")

with sc:
    display(plot.show())

RFBOutputContext()

In [20]:
plot.close()
sc.close()

## neuro data example

In [3]:
# load in movie 
movie = np.load('./fpl-scipy2023-data/neural_data/rcm.npy')

# load in identified nuerons
contours = np.load('./fpl-scipy2023-data/neural_data/contours.npy', allow_pickle=True)

In [22]:
# for the image data and contours
cnmf_iw = fpl.ImageWidget(
    movie, 
    vmin_vmax_sliders=True, 
    grid_plot_kwargs={"size": (600, 500)},
    cmap="gray"
)

# stack the plots and show them in sidecar
sc = Sidecar(title="good/bad comps")

with sc:
    display(cnmf_iw.show())

RFBOutputContext()

In [23]:
cnmf_iw.vmin_vmax_sliders[0].value = (-5, 15)

### Colors based on classifier output

In [24]:
# indices of accepted_ixs components
accepted_ixs = np.load('./fpl-scipy2023-data/neural_data/good_ixs.npy')

In [25]:
classifier = np.zeros(len(contours), dtype=int)
classifier[accepted_ixs] = 1

classifier

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1,
       1])

In [26]:
# add contours to the plot within the widget
contours_graphic = cnmf_iw.gridplot[0,0].add_line_collection(
    contours, 
    cmap="Set1",
    cmap_values=classifier,
    thickness=3, 
    name="contours"
)

## Red: class 0 - rejected

## Blue: class 1 - accepted

### cmaps for quantitative metrics 

#### yet another measure, signal-to-noise measure

In [27]:
# load in SNR comps
snr_comps = np.load('./fpl-scipy2023-data/neural_data/SNR_comps.npy')
np.log10(snr_comps)

array([ 1.30693739,  0.98832133,  0.61094959,  0.60680478,  0.78905172,
        0.75528032,  0.07143519,  0.20501458,  0.82659848,  0.72253894,
        0.50721547,  0.06808205,  0.09170267,  0.16848598,  1.21936687,
        0.60382365,  0.46201108,  0.62335868,  0.31698763,  0.6426985 ,
        0.60684258,  0.5431985 ,  0.59940984,  0.64724762,  0.29722249,
        0.29662902,  0.61015591,  0.31969486,  1.19550207,  0.5710063 ,
        0.46383772,  1.02528406,  0.72630303,  0.40776261,  0.19654701,
        0.52004469,  0.09369996, -0.14752256,  1.05310796,  0.37688607,
        0.41622687,  1.17317543,  0.73611689,  0.42182707,  0.3150909 ,
        0.21921744,  0.3789719 , -0.06830937,  0.35099327, -0.08058595,
        0.71089304,  0.44862497,  0.34431778,  0.03256675,  0.28882971,
        0.61887717,  0.76572043,  0.51032459,  0.63065994,  0.95790937,
        0.63282506,  0.65043055,  0.44677766,  0.31964022,  0.41866778,
        0.18621914,  0.90540626,  0.7797958 ,  0.3603792 ,  0.51

In [29]:
contours_graphic.cmap = "spring"
contours_graphic.cmap_values = np.log10(snr_comps)

## low signal to noise: purple

## high signal to noise: yellow

In [30]:
cnmf_iw.gridplot.close()
sc.close()

## Line Stack - useful for time series data

### 3 Generate some sine data, 3 million points

In [50]:
xs = np.linspace(0, 2_000, 3_000)
# sine wave
ys = np.sin(xs) * 20

data = np.vstack([ys] * 1_000)

In [51]:
data.shape

(1000, 3000)

In [52]:
plot = fpl.Plot(size=(600, 500), canvas="jupyter")

# line stack takes all the same arguments as line collection
# and behaves similarly
plot.add_line_stack(data, cmap="jet", separation=15, name="many-lines")

sc = Sidecar(title="line stack")

with sc:
    display(plot.show(maintain_aspect=False))

RFBOutputContext()

## Performant changes

In [53]:
plot["many-lines"].cmap = "viridis"

**more numpy-like indexing :D**

Set the cmap of the first 500 individual lines to `plasma`. 
This sets the cmap _along the line_

In [54]:
plot["many-lines"][:500].cmap = "plasma"

Every 5th datapoint to "white" or "black"

In [55]:
plot["many-lines"][:500].colors[::5] = "w"

In [49]:
plot.close()
sc.close()

## Linear Region Selector

Creates a linear region bounded graphic which can be moved along either the x-axis or y-axis.
Allows sub-selecting data from a `Graphic` or from multiple `Graphic` objects.

In [38]:
def interpolate(subdata: np.ndarray, axis: int):
    """1D interpolation to display within the preallocated data array"""
    x = np.arange(0, zoomed_prealloc)
    xp = np.linspace(0, zoomed_prealloc, subdata.shape[0])
    
    # interpolate to preallocated size
    return np.interp(x, xp, fp=subdata[:, axis])  # use the y-values

In [39]:
# data to plot
xs = np.linspace(0, 100, 1_000)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

plot = fpl.GridPlot((5, 1), size=(600, 600))

# preallocated size for zoomed data
zoomed_prealloc = 1_000
zoomed_init = np.column_stack([np.arange(zoomed_prealloc), np.random.rand(zoomed_prealloc)])

# sines and cosines
sines = [sine] * 2
cosines = [cosine] * 2

# make line stack
line_stack = plot[0, 0].add_line_stack(sines + cosines, separation=50)

# make selector
selector = line_stack.add_linear_region_selector()

# populate subplots with preallocated graphics
for i, subplot in enumerate(plot):
    if i == 0:
        # skip the first one
        continue
    # make line graphics for displaying zoomed data
    subplot.add_line(zoomed_init, name="zoomed")


def update_zoomed_subplots(ev):
    """update the zoomed subplots"""
    zoomed_data = selector.get_selected_data()
    
    for i in range(len(zoomed_data)):
        data = interpolate(zoomed_data[i], axis=1)
        plot[i + 1, 0]["zoomed"].data = data
        plot[i + 1, 0].auto_scale()


selector.selection.add_event_handler(update_zoomed_subplots)
plot.show()
sc = Sidecar(title="linear region selector")

with sc: 
    display(plot.show())

RFBOutputContext()

In [40]:
plot.close()
sc.close()

## Heatmap

In [4]:
temporal = np.load('./fpl-scipy2023-data/neural_data/temporal.npy')

In [11]:
# create plot
plot = fpl.GridPlot(shape=(1,2), names=[["heatmap", "timeseries"]])

def normalize(array):
    out = np.zeros(array.shape, dtype=array.dtype)
    for i in range(array.shape[0]):
        a = array[i]
        out[i] = ((a - np.min(a)) / (np.max(a - np.min(a))))
        
    return out

temporal_norm = normalize(temporal)
        
# add temporal traces as heatmap
heatmap = plot[0,0].add_heatmap(temporal_norm)

# add temporal traces as line stack
temporal_stack = plot[0,1].add_line_stack(temporal, colors="magenta")

for subplot in plot:
    subplot.camera.maintain_aspect = False

# heatmap support the same selectors as lines!
heatmap_ls = heatmap.add_linear_selector()
temporal_stack_ls = temporal_stack.add_linear_selector()
temporal_stack_ls.position_z = 10

# the corresponding imaging data and some callbacks to update frames
cnmf_iw = fpl.ImageWidget(movie, vmin_vmax_sliders=True, cmap="gray")

# update the linear selectors with current frame index
def update_linear_selector(change):
    ix = change["new"]
    
    heatmap_ls.selection = ix
    temporal_stack_ls.selection = ix
    
def update_ipywidget(ev):
    ix = ev.pick_info["selected_index"]
    # for line collection sends as list
    if not isinstance(ix, int):
        ix = ix[0]
    cnmf_iw.sliders["t"].value = ix
    
cnmf_iw.sliders["t"].observe(update_linear_selector, "value")
heatmap_ls.selection.add_event_handler(update_ipywidget)
temporal_stack_ls.selection.add_event_handler(update_ipywidget)

    
sc = Sidecar(title="heatmap interactive")

with sc:
    display(VBox([cnmf_iw.show(), plot.show()]))

RFBOutputContext()

RFBOutputContext()

In [12]:
cnmf_iw.vmin_vmax_sliders[0].value = (-5, 15)

In [7]:
plot.close()
sc.close()